In [1]:
import os

os.chdir('../..')
import utils
import wandb

In [2]:
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbCallback

In [3]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.metrics import AUC

In [4]:
wandb.init(
    project="fyp-hearts-lenet",
    
    config={
        "activation": 'relu',
        "dropout": random.uniform(0.01, 0.8),
        "filters_1": 6,
        "filters_2": 16,
        "dense_1": 64,
        "dense_2": 32,
        "epochs": 30,
        "optimizer": "adam",
        "loss":"binary_crossentropy",
        "metrics": tf.keras.metrics.AUC(num_thresholds=1000000, multi_label=True)
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: agarwalla-mohit (fyp-hearts). Use `wandb login --relogin` to force relogin


In [5]:
outputfolder = os.getcwd() + '/output/'
datafolder = os.getcwd() + '/datasets/PTB-XL/'
sampling_rate = 100
task = 'priority'
experiment_name = 'exp4'

data, raw_labels = utils.load_dataset(datafolder, sampling_rate=sampling_rate)
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
data, labels, Y, _ = utils.select_data(data, labels, task, 0, outputfolder+experiment_name+'/data/')

In [6]:
X_test = data[labels.strat_fold == 10]
y_test = Y[labels.strat_fold == 10]

X_val = data[labels.strat_fold == 9]
y_val = Y[labels.strat_fold == 9]

X_train = data[labels.strat_fold <= 8]
y_train = Y[labels.strat_fold <= 8]

n_classes = y_train.shape[1]

print(f"This experiment has {n_classes} classes")

X_train = X_train[:,:,0]
X_test = X_test[:,:,0]
X_val = X_val[:,:,0]

This experiment has 43 classes


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, LeakyReLU
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  Input, Reshape
from tensorflow.keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D,MaxPool1D,AveragePooling1D
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
import numpy as np
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential

In [8]:
config = wandb.config

lenet_5_model=Sequential()
lenet_5_model.add(Conv1D(filters=config.filters_1, kernel_size=3, padding='same', input_shape=(1000,1)))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(Conv1D(filters=config.filters_2, strides=1, kernel_size=5))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(GlobalAveragePooling1D())
lenet_5_model.add(Dense(config.dense_1, activation='relu'))
lenet_5_model.add(Dense(config.dense_1, activation='relu'))
lenet_5_model.add(Dense(n_classes, activation = 'sigmoid'))

In [9]:
lenet_5_model.compile(optimizer=config.optimizer,
                      loss=config.loss,
                      metrics=tf.keras.metrics.AUC(num_thresholds=100, multi_label=True))

In [10]:
history = lenet_5_model.fit(X_train, y_train, epochs=config.epochs, validation_data=(X_val, y_val),
                            callbacks=[
                                WandbMetricsLogger(log_freq=2),
                                WandbModelCheckpoint("models")
                            ])

wandb.finish()

Epoch 1/30
534/534 [==============================] - ETA: 0s - loss: 0.1527 - auc_1: 0.5133

INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.2s


534/534 [==============================] - 35s 54ms/step - loss: 0.1527 - auc_1: 0.5133 - val_loss: 0.1238 - val_auc_1: 0.5491
Epoch 2/30
534/534 [==============================] - ETA: 0s - loss: 0.1145 - auc_1: 0.5926

INFO:tensorflow:Assets written to: models\assets


...

534/534 [==============================] - 27s 50ms/step - loss: 0.0961 - auc_1: 0.7141 - val_loss: 0.1187 - val_auc_1: 0.6342
Epoch 30/30
533/534 [============================>.] - ETA: 0s - loss: 0.0960 - auc_1: 0.7152

INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.2s


534/534 [==============================] - 27s 50ms/step - loss: 0.0960 - auc_1: 0.7152 - val_loss: 0.1003 - val_auc_1: 0.6982


batch/auc_1,▄▄▅▆▅▇▇▇▇▇▇▇▇▆▇▇▇▇██▇▆███▇████▃██▇████▁█
batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▄▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁
epoch/auc_1,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████████
epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_auc_1,▂▄▁▂▅▅▁▂▃▂▇▂▆▇▅▇▂▅▅▇▃█▅▂▅█▇▄▅▇
epoch/val_loss,▂▅▄▃▁▁█▄▂▄▁▆▄▁▁▁▄▃▃▂▃▁▃▄▃▁▂▂▂▁
batch/auc_1,0.71521


In [11]:
from sklearn.metrics import roc_auc_score

preds = lenet_5_model.predict(X_test)
print(roc_auc_score(y_test,preds))

68/68 [==============================] - 1s 10ms/step
0.7716123980144007


## Try the sweep

In [8]:
sweep_config = {
    'method': 'random',  # Method of sweep: 'grid', 'random' or 'bayes'
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'filters_1': {
            'values': [16, 32, 64]
        },
        'filters_2': {
            'values': [32, 64, 128]
        },
        'dense_1': {
            'values': [64, 128, 256]
        },
        'learning_rate': {
            'values': [0.001, 0.0001]
        }
    }
}

# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project='lenet_5_sweep')

Create sweep with ID: pj0b3bw9
Sweep URL: https://wandb.ai/fyp-hearts/lenet_5_sweep/sweeps/pj0b3bw9


In [9]:
def train():
    # Initialize a new wandb run
    wandb.init()
    config = wandb.config

    # Define the model
    lenet_5_model = Sequential()
    lenet_5_model.add(Conv1D(filters=config.filters_1, kernel_size=3, padding='same', input_shape=(1000,1)))
    lenet_5_model.add(BatchNormalization())
    lenet_5_model.add(Activation('relu'))
    lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
    lenet_5_model.add(Conv1D(filters=config.filters_2, strides=1, kernel_size=5))
    lenet_5_model.add(BatchNormalization())
    lenet_5_model.add(Activation('relu'))
    lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
    lenet_5_model.add(GlobalAveragePooling1D())
    lenet_5_model.add(Dense(config.dense_1, activation='relu'))
    lenet_5_model.add(Dense(config.dense_1, activation='relu'))
    lenet_5_model.add(Dense(n_classes, activation='sigmoid'))

    # Compile the model
    lenet_5_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate), 
                          loss='binary_crossentropy', 
                          metrics=['accuracy', AUC(multi_label=True)])


    # Train the model
    lenet_5_model.fit(X_train, y_train, 
                      validation_data=(X_val, y_val),
                      epochs=30,
                      callbacks=[WandbCallback()])

# Execute the sweep
wandb.agent(sweep_id, train, count=20)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 0nwu1imm with config:
wandb: 	dense_1: 256
wandb: 	filters_1: 32
wandb: 	filters_2: 64
wandb: 	learning_rate: 0.0001


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/30
534/534 [==============================] - ETA: 0s - loss: 0.1909 - accuracy: 0.0544 - auc: 0.4925

INFO:tensorflow:Assets written to: c:\Mohit\Imperial\fyp-hearts\wandb\run-20240519_171001-0nwu1imm\files\model-best\assets


INFO:tensorflow:Assets written to: c:\Mohit\Imperial\fyp-hearts\wandb\run-20240519_171001-0nwu1imm\files\model-best\assets
wandb: Adding directory to artifact (c:\Mohit\Imperial\fyp-hearts\wandb\run-20240519_171001-0nwu1imm\files\model-best)... Done. 0.1s


...

wandb: ERROR Run l32dk5wf errored: FileNotFoundError(2, 'No such file or directory')
wandb: Agent Starting Run: ndfh95nx with config:
wandb: 	dense_1: 256
wandb: 	filters_1: 16
wandb: 	filters_2: 128
wandb: 	learning_rate: 0.001


Epoch 1/30


wandb: ERROR Run ndfh95nx errored: FileNotFoundError(2, 'No such file or directory')


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x0000029C1EC834F0>> (for post_run_cell):


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [ ]:
config = wandb.config

lenet_5_model = Sequential()
lenet_5_model.add(Conv1D(filters=config.filters_1, kernel_size=3, padding='same', input_shape=(1000,1)))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(Conv1D(filters=config.filters_2, strides=1, kernel_size=5))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(GlobalAveragePooling1D())
lenet_5_model.add(Dense(config.dense_1, activation='relu'))
lenet_5_model.add(Dense(config.dense_1, activation='relu'))
lenet_5_model.add(Dense(n_classes, activation='sigmoid'))

# Compile the model
lenet_5_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate), 
                        loss='binary_crossentropy', 
                        metrics=['accuracy', AUC()])


# Train the model
lenet_5_model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val),
                    epochs=30,
                    callbacks=[WandbCallback()])

Error: You must call wandb.init() before wandb.config.filters_1